In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [12]:
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [2]:
# Load the Quora dataset
data = pd.read_csv(r'C:\Users\HP\Documents\IITK\DL_Project_Submission\Project 3\train.csv')

In [3]:
data.shape

(1306122, 3)

In [7]:
data.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [4]:
df=pd.DataFrame(data)

In [5]:
df = df.drop_duplicates().reset_index(drop=True)
df = df.dropna().reset_index(drop=True)

In [8]:
df['target'] = np.where(df['target'] == 1, 'insincere', 'sincere')

In [9]:
df


,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,sincere
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",sincere
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,sincere
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,sincere
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,sincere
...,...,...,...
1306117,ffffcc4e2331aaf1e41e,What other technical skills do you need as a c...,sincere
1306118,ffffd431801e5a2f4861,Does MS in ECE have good job prospects in USA ...,sincere
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,sincere
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,sincere


In [10]:
# Tokenize the questions
tokenizer = Tokenizer(num_words=5000, lower=True)
tokenizer.fit_on_texts(df['question_text'])
X = tokenizer.texts_to_sequences(df['question_text'])
X = pad_sequences(X, maxlen=100)


In [13]:
# Split the dataset into training and testing sets
y = pd.get_dummies(df['target']).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1044897, 100), (261225, 100), (1044897, 2), (261225, 2))

In [16]:
# Build and train the LSTM model
model = Sequential()
model.add(Embedding(5000, 128, input_length=X.shape[1]))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))

In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
32654/32654 [==============================] - 7764s 238ms/step - loss: 0.1260 - accuracy: 0.9517
Epoch 2/10
32654/32654 [==============================] - 8640s 265ms/step - loss: 0.1160 - accuracy: 0.9553
Epoch 3/10
32654/32654 [==============================] - 6332s 194ms/step - loss: 0.1124 - accuracy: 0.9567
Epoch 4/10
32654/32654 [==============================] - 8285s 254ms/step - loss: 0.1097 - accuracy: 0.9578
Epoch 5/10
32654/32654 [==============================] - 6256s 192ms/step - loss: 0.1078 - accuracy: 0.9587
Epoch 6/10
32654/32654 [==============================] - 6406s 196ms/step - loss: 0.1063 - accuracy: 0.9591
Epoch 7/10
32654/32654 [==============================] - 6364s 195ms/step - loss: 0.1049 - accuracy: 0.9601
Epoch 8/10
32654/32654 [==============================] - 6297s 193ms/step - loss: 0.1037 - accuracy: 0.9604
Epoch 9/10
32654/32654 [==============================] - 6249s 191ms/step - loss: 0.1030 - accuracy: 0.9606
Epoch 10/10
32654/3

In [19]:
loss, acc = model.evaluate(X_test, y_test, verbose=2)
print(f'Loss: {loss}, Accuracy: {acc}')

8164/8164 - 241s - loss: 0.1175 - accuracy: 0.9555 - 241s/epoch - 30ms/step
Loss: 0.1174905002117157, Accuracy: 0.955528736114502


In [20]:
y_pred = (model.predict(X_test) > 0.5).astype(int)

8164/8164 [==============================] - 224s 27ms/step


In [22]:
from sklearn.metrics import accuracy_score, classification_report

In [23]:
# Print classification report     
print(classification_report(y_test, y_pred, target_names=['Not Spam', 'Spam']))

              precision    recall  f1-score   support

    Not Spam       0.71      0.46      0.56     15856
        Spam       0.97      0.99      0.98    245369

   micro avg       0.96      0.96      0.96    261225
   macro avg       0.84      0.72      0.77    261225
weighted avg       0.95      0.96      0.95    261225
 samples avg       0.96      0.96      0.96    261225

